In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
%matplotlib inline
from tasks import wsd
import pandas as pd
from tqdm.auto import tqdm
from sklearn.metrics import classification_report
from utils.dataset_download import harvest_data_from_extended_senses
from utils.classificaton_utils import binarize, vectorize_target_expressions,cosine_similiarity
from sklearn.svm import LinearSVC
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier


In [6]:
tqdm.pandas()
lemma,pos = 'machine', "NN"
senses = {'machine_nn01-38474140'} # machine_nn01-38475772 machine_nn01-38475923 machine_nn01-38475835 machine_nn01-38474140
relations = ['seed','synonym'] # ,'descendant','sibling'
eval_mode = "lemma_etal" # lemma or lemma_etal
    

In [23]:
df_train, df_val, df_test = binarize(lemma,
                        pos,
                        senses, 
                        relations,
                        strict_filter=True,
                        start=1700,
                        end=2000,
                        filter_val_by_year=True,
                        filter_test_by_year=True,
                        eval_mode=eval_mode)

# senses before filtering by date = 517
# senses after filtering by date = 433


# of seed senses 26 
# of synonyms 383 
# of branch senses 0


# of seeds selected 1 
# of synonyms selected 44 
# of branches selected 0
[LOG] 2905 quotations selected
[LOG] train = 2136 val = 292 test = 477 quotations


In [24]:

vector_col = "vector_bert_base_-1,-2,-3,-4_mean"

In [25]:

df = pd.concat([df_train,df_val], axis=0)
X,y = list(df[vector_col].values), list(df.label.values)

In [26]:
svm_model = LinearSVC(random_state=0, C=.1, tol=1e-5,class_weight='balanced')
svm_model.fit(X,y)
y_pred = wsd.clf_svm(vector_col,df_test, svm_model)
print(classification_report(y_pred,list(df_test.label.values)))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96       443
           1       0.45      0.41      0.43        34

    accuracy                           0.92       477
   macro avg       0.70      0.69      0.69       477
weighted avg       0.92      0.92      0.92       477



In [27]:

perc_model = Perceptron(validation_fraction=.2, early_stopping=True,class_weight='balanced')
perc_model.fit(X,y)
y_pred = wsd.clf_perceptron(vector_col,df_test, perc_model)
print(classification_report(y_pred,list(df_test.label.values)))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96       438
           1       0.61      0.49      0.54        39

    accuracy                           0.93       477
   macro avg       0.78      0.73      0.75       477
weighted avg       0.93      0.93      0.93       477



In [28]:
mlperc_model = MLPClassifier(validation_fraction=.2, early_stopping=True, solver='lbfgs',activation='relu')
mlperc_model.fit(X,y)
y_pred = wsd.clf_perceptron(vector_col,df_test, mlperc_model)
print(classification_report(y_pred,list(df_test.label.values)))

              precision    recall  f1-score   support

           0       0.97      0.96      0.97       450
           1       0.45      0.52      0.48        27

    accuracy                           0.94       477
   macro avg       0.71      0.74      0.72       477
weighted avg       0.94      0.94      0.94       477

